# Парсинг сайта https://tgads.su

In [33]:
!pip3 install pyppeteer
!pip3 install openpyxl
!pip3 install pandas

You should consider upgrading via the '/Users/syper_olao/Desktop/program/programs/work/selenium/.venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/syper_olao/Desktop/program/programs/work/selenium/.venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/syper_olao/Desktop/program/programs/work/selenium/.venv/bin/python3 -m pip install --upgrade pip' command.


In [34]:
from pyppeteer import launch
from time import sleep

### Connection to environment

In [35]:
import os
from pathlib import Path
from dotenv import load_dotenv, dotenv_values

dotenv_path = os.path.join(Path(os.path.abspath('')).parent.parent, ".env")

if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    
config_env = os.environ if not os.path.exists(dotenv_path) else dotenv_values(".env")

### Запуск браузера 

1. В режиме селениума с демонстрации действий
2. В режиме движка

In [36]:
browser = await launch(headless=False)
# browser = await launch(
#             headless=True, args=["--no-sandbox", "--disable-setuid-sandbox"]
#         )
page = await browser.newPage()


### Логин на сайт

In [37]:
async def login_page():
    await page.goto("https://tgads.su/auth")
    await page.click("#email")
    await page.type("#email", config_env["TGADS_LOGIN"])
    
    await page.click("#password")
    await page.type("#password", config_env["TGADS_PASSWORD"])
    sleep(1)
    await page.click('button[type="button"]')
    await page.waitForNavigation()
    sleep(1)


await login_page()

### Функция запроса браузером данных по постам

In [38]:
async def request_posts_data(data_from, data_to):
    data = await page.evaluate(f'''
            async () => {{
                try {{
                    const res = await fetch("https://tgads.su/cp/get?date_from={data_from}&date_to={data_to}&table=ads", {{
                        credentials: 'include',
                    }});
                    if (!res.ok) {{
                        throw new Error(`HTTP error! status: ${{res.status}}`);
                    }}
                    const data = await res.json();
                    return data; 
                }} catch (error) {{
                    return {{error: error.toString()}}; 
                }}
            }}
        ''')
    return data

### Запрос от по дате

date_from - дата начала

date_to - дата завершения

In [39]:
date_from="2024-06-01"
date_to="2024-07-25"
fetched_data = await request_posts_data(date_from, date_to)
array_of_posts = fetched_data['data']


### Запрос дополнительной информации по каждому посту

In [40]:
async def request_post_data_by_id(id):
    data = await page.evaluate(
        f"""
            async () => {{
                try {{
                   const res = await fetch("https://tgads.su/ad/preview/{id}",{{
                    credentials: 'include',
                    headers: {{
                        "Content-Type": "application/json",
                    }},
                   }});
                    if (!res.ok) {{
                        throw new Error(`HTTP error! status: {{res.status}}`);
                    }}
                    const data = await res.json();
                    return data; 
                }} catch (error) {{
                    return {{error: error.toString()}}; 
                }}
            }}
        """
    )
    return data

### Необходимые заголовки и их индексы

In [41]:
headers = {
    "ad_id": 1,
    "ad_title": 2,
    "name_cab": 3,
    "channel_id": 4,
    "text": 5,
    "status": 6,
    "Spend": 7,
    "Joined": 8,
    "Impressions": 9,
    "Clicks": 10,
    "CPF": 11,
    "CPM": 12,
    "Budget": 13,
    "date_start": 14,
    "date_end": 15,
}

### Работа с excel: создание столбцов из header

In [42]:
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Data"

headers_arr = list(headers.keys())
for i in range(len(headers_arr)):
    ws.cell(row=i+1, column=1, value=headers_arr[i])

### Получение информации о канале

In [43]:
async def request_post_channel(id):
    fetched_data_2 = await request_post_data_by_id(id)
    post_info = fetched_data_2["data"]["ad"]
    channel = post_info.get("channel", {})
    if isinstance(channel, list):
        channel = {}
    return channel, post_info

### Запись данных в ячейку, при отсутсвии записываем '-'

In [44]:
add_to_cell = lambda row, column, value: ws.cell(
    row=row, column=column, value=value if value is not None else "-"
)

In [45]:
def add_object_to_excel(obj, column):
    for key in obj:
        add_to_cell(key, column, obj[key])

In [46]:
for index, post in enumerate(array_of_posts):

    channel, post_info = await request_post_channel(post["system_ad_id"])

    column = index + 2

    obj = {
        headers["ad_id"]: post.get("ad_id", None),
        headers["ad_title"]: post.get("ad_title", None),
        headers["name_cab"]: channel.get("title", None),
        headers["channel_id"]: post.get("ad_promote_url", None),
        headers["text"]: post_info.get("text", None),
        headers["status"]: post.get("ad_status_text", None),
        headers["Spend"]: post.get("spent_budget", None),
        headers["Joined"]: post.get("joins", None),
        headers["Impressions"]: post.get("views", None),
        headers["Clicks"]: post.get("clicks", None),
        headers["CPF"]: post.get("cpf", None),
        headers["CPM"]: post.get("cpm", None),
        headers["Budget"]: post.get("budget", None),
        headers["date_start"]: post.get("ad_created_at", None),
        headers["date_end"]: " ",
    }
    add_object_to_excel(obj, column)
  

### Сохраняем файл, закрываем браузер

In [47]:
wb.save("post_data.xlsx")
# await browser.close()

In [48]:
import pandas as pd
post_data = pd.read_excel('post_data.xlsx').T
post_data.columns = post_data.iloc[0]
post_data = post_data[1:]
post_data = post_data.reset_index().rename(columns={'index':'ad_id'}).astype(str)
post_data.to_excel('post_data_T.xlsx', index=False)
post_data

ad_id,ad_id,ad_title,name_cab,channel_id,text,status,Spend,Joined,Impressions,Clicks,CPF,CPM,Budget,date_start,date_end
0,48,// внести в 1ф,На Своих Двоих🎒,https://t.me/getaroundrussia,Путешествую по великой России на канале @getar...,Остановлено,1200.17,0,4000,5,0,300.04,0,2024-07-18 14:40,
1,47,// внести в 1ф,На Своих Двоих🎒,https://t.me/getaroundrussia,Путешествую по великой России на канале @getar...,Остановлено,1200.17,1,4000,8,1200.17,300.04,0,2024-07-18 14:40,
2,46,// внести в 1ф,На Своих Двоих🎒,https://t.me/getaroundrussia,Путешествую по великой России на канале @getar...,Остановлено,1200.17,0,4000,4,0,300.04,0,2024-07-18 14:39,
3,44,// внести в 1ф,На Своих Двоих🎒,https://t.me/getaroundrussia,Путешествую по великой России на канале @getar...,Остановлено,1200.17,0,4000,4,0,300.04,0,2024-07-18 14:39,
4,45,// внести в 1ф,На Своих Двоих🎒,https://t.me/getaroundrussia,Путешествую по великой России на канале @getar...,Остановлено,1200.17,0,4000,1,0,300.04,0,2024-07-18 14:39,
5,43,// внести в 1ф,На Своих Двоих🎒,https://t.me/getaroundrussia,Путешествую по великой России на канале @getar...,Остановлено,1200.17,1,4000,9,1200.17,300.04,0,2024-07-18 12:45,
6,42,внести в 1ф //,На Своих Двоих🎒,https://t.me/getaroundrussia,Путешествую по России по низким ценам. Делюсь ...,Неактивно,1199.87,0,3999,10,0,300.04,0,2024-07-17 16:55,
7,41,внести в 1ф //,На Своих Двоих🎒,https://t.me/getaroundrussia,Путешествуем по России на канале @getaroundrus...,Неактивно,1086.45,0,3621,17,0,300.04,0,2024-07-16 14:30,
8,34,#3704837,Мир Будущего,https://t.me/digit_future,Откройте дверь в будущее: последние достижения...,Остановлено,999.9,0,3331,13,0,300.18,0,2024-06-24 16:11,
9,33,#3704836,Мир Будущего,https://t.me/digit_future,"Погрузитесь в мир, где технологии формируют бу...",Остановлено,1999.8,6,6662,21,333.3,300.18,0,2024-06-24 16:11,
